In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split, cross_val_score, StratifiedKFold
from sklearn import grid_search
from sklearn import linear_model, metrics, preprocessing
from sklearn.feature_selection import RFE,SelectPercentile,f_classif,chi2
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
import xgboost as xgb

### Load data

In [2]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

skf_cv = StratifiedKFold(df_train['TARGET'], n_folds=6, shuffle=True, random_state=None) # split target equally
def score_model(model,X,t):
    #return cross_val_score(model, X, t, cv=6, scoring="roc_auc",n_jobs=3)
    return cross_val_score(model, X, t, cv=skf_cv, scoring="roc_auc",n_jobs=3)

### Remove constant columns

In [3]:
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

### Remove duplicated columns

In [4]:
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

### Other cleaning

In [5]:
# Replace -999999 in var3 column with most common value 2 
df_train = df_train.replace(-999999,2)
df_test = df_test.replace(-999999,2)

### Feature selection

In [6]:
Xdf_tr = df_train.drop('TARGET', 1)
tdf_tr = df_train.TARGET

from sklearn.preprocessing import Binarizer, scale

p = 30 # percentage of selected features

X_bin = Binarizer().fit_transform(scale(Xdf_tr))
selectChi2 = SelectPercentile(chi2, percentile=p).fit(X_bin, tdf_tr)
selectF_classif = SelectPercentile(f_classif, percentile=p).fit(Xdf_tr, tdf_tr)

chi2_selected = selectChi2.get_support()
chi2_selected_features = [ f for i,f in enumerate(Xdf_tr.columns) if chi2_selected[i]]

f_classif_selected = selectF_classif.get_support()
f_classif_selected_features = [ f for i,f in enumerate(Xdf_tr.columns) if f_classif_selected[i]]


selected = chi2_selected | f_classif_selected
print('Chi2 & F_classif selected {} features'.format(selected.sum()))
features = [ f for f,s in zip(Xdf_tr.columns, selected) if s]

print('List of features:{}'.format(features))
Xdf=df_train[features]

Chi2 & F_classif selected 106 features
List of features:['var15', 'imp_op_var40_efect_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var41_efect_ult1', 'imp_op_var41_efect_ult3', 'imp_op_var41_ult1', 'imp_op_var39_efect_ult1', 'imp_op_var39_efect_ult3', 'imp_op_var39_ult1', 'ind_var5_0', 'ind_var5', 'ind_var8_0', 'ind_var8', 'ind_var12_0', 'ind_var12', 'ind_var13_0', 'ind_var13_corto_0', 'ind_var13_corto', 'ind_var13_largo_0', 'ind_var13_largo', 'ind_var13', 'ind_var14_0', 'ind_var24_0', 'ind_var24', 'ind_var25_cte', 'ind_var26_0', 'ind_var26_cte', 'ind_var25_0', 'ind_var30', 'ind_var39_0', 'ind_var41_0', 'num_var4', 'num_var5_0', 'num_var5', 'num_var8_0', 'num_var8', 'num_var12_0', 'num_var12', 'num_var13_0', 'num_var13_corto_0', 'num_var13_corto', 'num_var13_largo_0', 'num_var13_largo', 'num_var13', 'num_var14_0', 'num_var24_0', 'num_var24', 'num_var26_0', 'num_var25_0', 'num_var30_0', 'num_var30', 'num_var35', 'num_var39_0', 'num_var41_0', 'num_var42_0', 'num_var42', 'saldo_var5', 'saldo

/home/nguyen/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


### Extract training data

In [7]:
#from sklearn import preprocessing
#X_tr = preprocessing.scale(Xdf.as_matrix())

X_tr = Xdf.as_matrix()
t_tr = df_train['TARGET'].as_matrix()

N_tr=np.shape(X_tr)[0]
D=np.shape(X_tr)[1]

In [51]:
# classifier
# http://xgboost.readthedocs.org/en/latest/parameter.html
# http://xgboost.readthedocs.org/en/latest/param_tuning.html
clf = xgb.XGBClassifier(
      objective= 'binary:logistic', 
      learning_rate=0.02, 
      n_estimators=200,
      max_depth=5,
      subsample=0.85,
      colsample_bytree=0.85,
      nthread=3,
      silent = 1,
      seed=8
)

cv_scores=score_model(clf, X_tr,t_tr)\

In [53]:
print("\nModel Report")
print("AUC Score (cross-validation): %f (+- %f)" % (cv_scores.mean(), cv_scores.std()))


Model Report
AUC Score (cross-validation): 0.838992 (+- 0.005478)


In [ ]:
clf = xgb.XGBClassifier(
      objective= 'binary:logistic', 
      n_estimators=500,
      nthread=3,
      silent = 1,
      seed=8
)
parameters = {
    'max_depth': [6,7,8],
    'learning_rate': [0.015, 0.0175, 0.02],
    'subsample': [0.85, 0.9],
    'colsample_bytree': [0.85, 0.9]
}

clf = grid_search.GridSearchCV(clf, parameters, n_jobs=3, cv=skf_cv, scoring='roc_auc')
clf.fit(X_tr, t_tr)

In [ ]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
print clf.grid_scores_